In [ ]:
import os
from bs4 import BeautifulSoup
import pandas as pd

def parse_html_file(filepath):
    with open(filepath, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")
    
    tables = soup.find_all("table")
    records = []

    for table in tables:
        business_name = table.find("th").text.strip()
        rows = table.find_all("tr")[1:]
        data = {'business_name': business_name}
        for row in rows:
            cells = row.find_all("td")
            for i in range(0, len(cells), 2):
                key = cells[i].text.strip().lower()
                value = cells[i+1].text.strip() if i+1 < len(cells) else None
                data[key] = value
        records.append(data)
    
    return pd.DataFrame(records)

html_dir = "."
html_files = [f for f in os.listdir(html_dir) if f.endswith(".html")]
all_businesses_df = pd.concat([parse_html_file(os.path.join(html_dir, file)) for file in html_files], ignore_index=True)


In [ ]:
# Let's define some reusable Python functions for the following:
# 1. Handling Parquet files
# 2. Reading and summarizing PDF files
# 3. Connecting to and querying SQLite databases
# 4. Creating data visualizations from DataFrames

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import PyPDF2
import pyarrow.parquet as pq
import io

# Function to read and display Parquet file data
def read_parquet_file(file_path):
    df = pd.read_parquet(file_path)
    print("Parquet File Loaded Successfully")
    print("Shape:", df.shape)
    print("Head:")
    print(df.head())
    return df

# Function to visualize basic statistics of a DataFrame
def visualize_dataframe(df, numerical_only=True):
    if numerical_only:
        df = df.select_dtypes(include='number')

    sns.set(style='whitegrid')
    for column in df.columns:
        plt.figure(figsize=(8, 4))
        sns.histplot(df[column].dropna(), kde=True, bins=30)
        plt.title(f'Distribution of {column}')
        plt.xlabel(column)
        plt.ylabel('Frequency')
        plt.tight_layout()
        plt.show()

# Function to read and extract text from a PDF file
def extract_text_from_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() or ''
    return text.strip()

# Function to connect to a SQLite DB and run a sample query
def query_sqlite_db(db_path, query="SELECT name FROM sqlite_master WHERE type='table';"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    conn.close()
    return results

In [ ]:
# Web Scraping and Data Cleaning from Local HTML Files

import os
import pandas as pd
from bs4 import BeautifulSoup

# 1. Define a function to extract business data from a single HTML file
def extract_business_data_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    tables = soup.find_all('table')

    records = []
    for table in tables:
        business = {}
        header = table.find('th')
        if header:
            business['business_name'] = header.text.strip()
        rows = table.find_all('tr')[1:]
        for row in rows:
            cells = row.find_all('td')
            for i in range(0, len(cells), 2):
                key = cells[i].text.strip()
                value = cells[i + 1].text.strip()
                business[key] = value
        records.append(business)

    return records

# 2. Aggregate data from all HTML files in the current directory
html_data = []
html_files = [f for f in os.listdir() if f.endswith('.html') and 'biz' in f]

for filename in html_files:
    with open(filename, encoding='utf-8') as file:
        content = file.read()
        html_data.extend(extract_business_data_from_html(content))

# 3. Create DataFrame from the scraped data
df = pd.DataFrame(html_data)

# 4. Clean the DataFrame
# Convert "null" strings to actual NaNs, remove duplicates
clean_df = df.replace({'null': pd.NA}).drop_duplicates()

# Convert data types where applicable
cols_to_convert = ['business_id', 'latitude', 'longitude', 'business_certificate', 'owner_zip']
for col in cols_to_convert:
    clean_df[col] = pd.to_numeric(clean_df[col], errors='coerce')

# 5. Show data summary and sample
print("\n--- Data Overview ---")
print(clean_df.info())
print("\n--- Sample Data ---")
print(clean_df.head())

# Optional: Save to CSV for analysis or ML tasks
clean_df.to_csv('business_data_cleaned.csv', index=False)

In [ ]:
def clean_data(df):
    df = df.drop_duplicates()
    df = df.replace({'null': pd.NA, '': pd.NA})
    df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]
    
    for col in ['latitude', 'longitude']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    return df

cleaned_df = clean_data(all_businesses_df)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_tax_code_distribution(df):
    plt.figure(figsize=(12, 6))
    sns.countplot(y="tax_code", data=df, order=df['tax_code'].value_counts().index)
    plt.title("Distribution of Business Tax Codes")
    plt.xlabel("Count")
    plt.ylabel("Tax Code")
    plt.tight_layout()
    plt.show()

def plot_business_locations(df):
    plt.figure(figsize=(10, 6))
    valid_coords = df.dropna(subset=["latitude", "longitude"])
    plt.scatter(valid_coords["longitude"], valid_coords["latitude"], alpha=0.6)
    plt.title("Business Locations (Lat/Lon)")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
import pyarrow.parquet as pq

def read_parquet_file(filepath):
    table = pq.read_table(filepath)
    return table.to_pandas()


In [ ]:
import PyPDF2

def extract_text_from_pdf(filepath):
    with open(filepath, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text


In [ ]:
import sqlite3

def load_sqlite_db(db_path):
    conn = sqlite3.connect(db_path)
    df_list = {}
    for table_name in pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)["name"]:
        df_list[table_name] = pd.read_sql(f"SELECT * FROM {table_name}", conn)
    conn.close()
    return df_list


In [ ]:
def export_cleaned_data(df, filename="cleaned_businesses.csv"):
    df.to_csv(filename, index=False)
